## Problem statement : Predict total fare for each flight for all carriers on a given day.

In [40]:
!pip install xgboost lightgbm


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import pandas as pd
import numpy as np
import re
import warnings
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,StackingRegressor,VotingRegressor,HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso,ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import TransformedTargetRegressor
import numpy as np


warnings.filterwarnings('ignore')

In [42]:
# Loading the datasets

#Train Dataframe
service_index_df=pd.read_csv("training\\training\\service_index.csv",index_col=0)
train_fares_df=pd.read_csv("training\\training\\train_fares.csv",index_col=0,on_bad_lines='skip', engine='python')
train_schedules_df=pd.read_csv("training\\training\\train_schedules.csv",index_col=0)

#Test Dataframe
test_fares_df=pd.read_csv("test\\test\\test_fares_data.csv",index_col=0, engine='python')
test_schedules_df=pd.read_csv("test\\test\\test_schedules.csv",index_col=0)

In [43]:
mean_fare = train_fares_df['total_fare'].mean()
mean_fare

np.float64(680.2354181635692)

In [44]:
for df in [service_index_df,train_fares_df,train_schedules_df,test_fares_df,test_schedules_df]:
  print(df.duplicated().sum())
  print(df.isna().sum())
  df.drop_duplicates(inplace=True)

0
yr               0
mo               0
origin           0
destination      0
carrier          0
scaled_demand    0
scaled_share     0
dtype: int64
462578
origin              0
destination         0
carrier             0
flt_num             0
flt_departure_dt    0
observation_date    0
total_fare          0
origin_city         0
destination_city    0
dtype: int64
0
carrier                     0
flt_num                     0
origin                      0
destination                 0
flt_departure_dt            0
flt_departure_local_time    0
flt_arrival_local_time      0
flt_departure_gmt           0
flt_arrival_gmt             0
dtype: int64
0
origin              0
destination         0
carrier             0
flt_num             0
flt_departure_dt    0
observation_date    0
origin_city         0
destination_city    0
dtype: int64
0
carrier                     0
flt_num                     0
origin                      0
destination                 0
flt_departure_dt            0
flt_de

In [45]:
#Basic testing and cleaning to make sure I dont face any joining issues afterwards
def data_check(df):
    for col in df.select_dtypes(exclude='number').columns:
        print(f'''\n================================{[ col ]}====================================''')
        print(df[col].unique())
        print(df[col].isnull().sum())
data_check(test_schedules_df)


================================['carrier']====================================
['L1' 'OTH' 'U3' 'L2' 'U1' 'L3']
0

================================['origin']====================================
['Airport26' 'Airport17' 'Airport4' 'Airport20' 'Airport30' 'Airport31'
 'Airport60' 'Airport43']
0

================================['destination']====================================
['Airport30' 'Airport4' 'Airport17' 'Airport43' 'Airport31' 'Airport60'
 'Airport26' 'Airport20']
0

================================['flt_departure_dt']====================================
['2019-01-01' '2019-01-02' '2019-01-03' '2019-01-04' '2019-01-05'
 '2019-01-06' '2019-01-07']
0

================================['flt_departure_local_time']====================================
['2019-01-01 18:28:00.0' '2019-01-01 07:50:00.0' '2019-01-01 11:55:00.0'
 ... '2019-01-07 19:49:00.0' '2019-01-07 07:48:00.0'
 '2019-01-07 09:56:00.0']
0

================================['flt_arrival_local_time']======================

In [46]:
#Make date consistent in all
def make_date_consistent(df,sample_size=500):
    for col in df.select_dtypes(include='object'):
        sample = df[col].head(sample_size)
        if sample.empty:
            continue

        parsed = pd.to_datetime(sample, errors='coerce')
        if parsed.notna().all():
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d %H-%M-%S')
    return df


for df in [service_index_df,train_fares_df,train_schedules_df,test_fares_df,test_schedules_df]:
    df=make_date_consistent(df)

In [47]:
# Joining fares to schedules
def process_schedule_data(df):
  df['flight_duration']=abs((pd.to_datetime(df['flt_departure_local_time'])-pd.to_datetime(df['flt_arrival_local_time'])).dt.total_seconds()/60)
  df['tz']=abs((pd.to_datetime(df['flt_departure_local_time'])-pd.to_datetime(df['flt_departure_gmt'])).dt.total_seconds()/60)
  df = (
    df
    .groupby(['carrier', 'flt_num', 'origin', 'destination', 'flt_departure_dt'])
    .agg(
        flight_duration_mean=('flight_duration', 'mean'),
        tz_mean=('tz', 'mean'),
        num_flights=('flt_departure_local_time', 'count')
    )
    .reset_index())
  return df

train_schedules_df_grouped=process_schedule_data(train_schedules_df.copy())
test_schedules_df_grouped=process_schedule_data(test_schedules_df.copy())
df_train=pd.merge(train_fares_df,train_schedules_df_grouped,how='left',on=['carrier','flt_num','origin','destination','flt_departure_dt'])
df_test=pd.merge(test_fares_df,test_schedules_df_grouped,how='left',on=['carrier','flt_num','origin','destination','flt_departure_dt'])
df_train['mo']=pd.to_datetime(df_train['flt_departure_dt']).dt.month
df_test['mo']=pd.to_datetime(df_test['flt_departure_dt']).dt.month



In [48]:
service_mapping=service_index_df.groupby(
    ['mo','origin','destination','carrier']
).agg(
    {'scaled_demand':'mean',
     'scaled_share':'mean'}
)
df_train = df_train.merge(
    service_mapping,
    on=['mo','origin','destination','carrier'],
    how='left',
    validate='many_to_one'
)

df_test = df_test.merge(
    service_mapping,
    on=['mo','origin','destination','carrier'],
    how='left',
    validate='many_to_one'
)

In [49]:
display(test_fares_df.tail(2))
display(test_schedules_df.tail(2))
display(service_index_df.tail(2))
display(df_train.tail(2))

,origin,destination,carrier,flt_num,flt_departure_dt,observation_date,origin_city,destination_city
29824,Airport17,Airport4,L1,18471,2019-01-07 00-00-00,2018-12-31 00-00-00,City17,City4
29825,Airport17,Airport4,L1,1487,2019-01-07 00-00-00,2018-12-31 00-00-00,City17,City4


,carrier,flt_num,origin,destination,flt_departure_dt,flt_departure_local_time,flt_arrival_local_time,flt_departure_gmt,flt_arrival_gmt
1233,L2,8018,Airport43,Airport4,2019-01-07 00-00-00,2019-01-07 09-56-00,2019-01-07 12-47-00,2019-01-07 15-56-00,2019-01-07 18-47-00
1234,L1,5823,Airport43,Airport4,2019-01-07 00-00-00,2019-01-07 15-55-00,2019-01-07 18-51-00,2019-01-07 21-55-00,2019-01-08 00-51-00


,yr,mo,origin,destination,carrier,scaled_demand,scaled_share
18656,2018,12,Airport66,Airport43,L3,3374,168
18657,2018,12,Airport8,Airport20,U2,2600,279


,origin,destination,carrier,flt_num,flt_departure_dt,observation_date,total_fare,origin_city,destination_city,flight_duration_mean,tz_mean,num_flights,mo,scaled_demand,scaled_share
1697436,Airport20,Airport4,L1,7434,2018-10-21 00-00-00,2018-09-19 00-00-00,1428,City19,City4,120.0,240.0,1.0,10,2134.0,340.0
1697437,Airport20,Airport4,L1,5927,2018-10-21 00-00-00,2018-09-19 00-00-00,916,City19,City4,180.0,240.0,1.0,10,2134.0,340.0


In [50]:
def add_features(df):
    df = df.copy()

    dt = pd.to_datetime(df['flt_departure_dt'])
    ob_dt= pd.to_datetime(df['observation_date'])
    df['month'] = dt.dt.month
    df['day_of_week'] = dt.dt.weekday            # 0=Mon, 6=Sun
    df['week_of_year'] = dt.dt.isocalendar().week.astype(int)
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_month_start'] = dt.dt.is_month_start.astype(int)
    df['is_month_end'] = dt.dt.is_month_end.astype(int)

    df['route'] = df['origin'] + '_' + df['destination']
    df['route_len'] = df['route'].str.len()      # weak but harmless

    df['flights_per_route_day'] = (
        df.groupby(['route', 'flt_departure_dt'])['flt_num']
          .transform('count')
    )
    df['carrier_flights_route_day'] = (
        df.groupby(['carrier', 'route', 'flt_departure_dt'])['flt_num']
          .transform('count')
    )
    df['carrier_share_route_day'] = (
        df['carrier_flights_route_day'] /
        df['flights_per_route_day'].replace(0, np.nan)
    )
    df['days_until_departure']= abs((dt - ob_dt).dt.days)
    df['flight_duration_log'] = np.log1p(df['flight_duration_mean'])
    df['tz_abs'] = df['tz_mean'].abs()
    df['origin_freq'] = df['origin'].map(df['origin'].value_counts(normalize=True))
    df['destination_freq'] = df['destination'].map(df['destination'].value_counts(normalize=True))
    df['route_freq'] = df['route'].map(df['route'].value_counts(normalize=True))
    df['demand_x_weekend'] = df['scaled_demand'] * df['is_weekend']
    df['share_x_congestion'] = (df['scaled_share'] * df['flights_per_route_day'])
    return df


df_train = add_features(df_train,)
df_test = add_features(df_test, )


In [51]:
for col in df_train.select_dtypes(include='number').columns:
  print(f'''{col} ===> {df_train[col].corr(df_train['total_fare'])}''')

flt_num ===> 0.0704999332759083
total_fare ===> 1.0
flight_duration_mean ===> 0.2909769303820728
tz_mean ===> -0.31123321420096745
num_flights ===> 0.0006751068657137381
mo ===> 0.0839997719023099
scaled_demand ===> -0.3185890839820095
scaled_share ===> 0.29397817987075764
month ===> 0.0839997719023099
day_of_week ===> 0.0970782434489908
week_of_year ===> 0.07853961535084965
is_weekend ===> 0.07864405986452762
is_month_start ===> -0.007012969710309297
is_month_end ===> 0.010059169175418995
route_len ===> -0.25693900287886956
flights_per_route_day ===> -0.26654329223737955
carrier_flights_route_day ===> -0.10524242141789111
carrier_share_route_day ===> 0.30717430449978
days_until_departure ===> -0.2622775400334569
flight_duration_log ===> 0.2857878924102893
tz_abs ===> -0.31123321420096745
origin_freq ===> -0.16248070103450912
destination_freq ===> -0.222028746889352
route_freq ===> -0.30772663114939314
demand_x_weekend ===> -0.0039070474158275
share_x_congestion ===> -0.209191720533925

In [ ]:
df_train.isna().sum().sort_values(ascending=False)

'''
scaled_demand                100308
scaled_share                 100308
demand_x_weekend             100308
share_x_congestion           100308
flight_duration_log           99108
tz_abs                        99108
tz_mean                       99108
num_flights                   99108
flight_duration_mean          99108

'''

def custom_impute(df_train):
    grouped_data=df_train.groupby(['origin','destination','carrier','flt_num']).agg({'flight_duration_mean':'mean','tz_mean':'mean','num_flights':'mean'})
    df_train=df_train.merge(grouped_data,how='left',on=['origin','destination','carrier','flt_num'],suffixes=('', '_grp'))
    df_train['flight_duration_mean'].fillna(df_train['flight_duration_mean_grp'],inplace=True)
    df_train['tz_mean'].fillna(df_train['tz_mean_grp'],inplace=True)
    df_train['num_flights'].fillna(df_train['num_flights_grp'],inplace=True)
    df_train.drop(['flight_duration_mean_grp','tz_mean_grp','num_flights_grp'],axis=1,inplace=True)

    grouped_data=df_train.groupby(['origin','destination','carrier']).agg({'flight_duration_mean':'mean','tz_mean':'mean','num_flights':'mean'})
    df_train=df_train.merge(grouped_data,how='left',on=['origin','destination','carrier'],suffixes=('', '_grp'))
    df_train['flight_duration_mean'].fillna(df_train['flight_duration_mean_grp'],inplace=True)
    df_train['tz_mean'].fillna(df_train['tz_mean_grp'],inplace=True)
    df_train['num_flights'].fillna(df_train['num_flights_grp'],inplace=True)
    df_train.drop(['flight_duration_mean_grp','tz_mean_grp','num_flights_grp'],axis=1,inplace=True)

    return df_train

df_train=custom_impute(df_train)
df_test=custom_impute(df_test)

scaled_demand                100308
scaled_share                 100308
demand_x_weekend             100308
share_x_congestion           100308
flight_duration_log           99108
tz_abs                        99108
tz_mean                       99108
num_flights                   99108
flight_duration_mean          99108
origin                            0
destination                       0
origin_city                       0
total_fare                        0
observation_date                  0
destination_city                  0
carrier                           0
flt_departure_dt                  0
flt_num                           0
week_of_year                      0
day_of_week                       0
month                             0
mo                                0
is_weekend                        0
is_month_start                    0
is_month_end                      0
route                             0
carrier_share_route_day           0
carrier_flights_route_day   

In [ ]:
# Drop columns that won't be used for modeling
# Keep missing values for imputation instead of dropping rows

df_train = df_train.drop(['flt_departure_dt','observation_date'], axis=1)
df_test = df_test.drop(['flt_departure_dt','observation_date'], axis=1)

df_train.shape, df_test.shape


((1697438, 32), (29825, 31))

In [ ]:
y=df_train.pop('total_fare')
X=df_train

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# Columns
num_cols = X.select_dtypes(include='number').columns
cat_cols = X.select_dtypes(exclude='number').columns

# Preprocess
numeric_pipe = Pipeline(steps=[
    ('scale',StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', min_frequency=50, sparse_output=True))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, num_cols),
        ('cat', categorical_pipe, cat_cols),
    ],
    remainder='passthrough'
)


pipe = Pipeline(steps=[
    ('prep', preprocess),
    ('model', KNeighborsRegressor()),
])

pipe


,steps,"[('prep', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
X_train[X_train.isna()].head()

,origin,destination,carrier,flt_num,origin_city,destination_city,flight_duration_mean,tz_mean,num_flights,mo,...,carrier_flights_route_day,carrier_share_route_day,days_until_departure,flight_duration_log,tz_abs,origin_freq,destination_freq,route_freq,demand_x_weekend,share_x_congestion
605378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1429016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1236311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Train 
tmp = pipe.fit(X_train, y_train)


ValueError: Input X contains NaN.
KNeighborsRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Evaluate

y_pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mae


136.7957763671875